## Imports

In [116]:
from bayes_opt import BayesianOptimization

In [2]:
import pandas as pd
import numpy as np
from datetime import timedelta
from tqdm import tqdm_notebook as tqdm
from sklearn import metrics
from sklearn.model_selection import StratifiedKFold
import lightgbm as lgb
from matplotlib import pyplot as plt
#import seaborn as sns
from collections import defaultdict, Counter
from sklearn.preprocessing import StandardScaler, MinMaxScaler, PolynomialFeatures, RobustScaler
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score as auc
from sklearn.linear_model import LogisticRegression
from scipy.special import logit

In [69]:
import lightgbm
from xgboost import XGBClassifier
from bayes_opt import BayesianOptimization
from sklearn.ensemble import RandomForestClassifier

In [118]:
import xgboost as xgb

In [82]:
file = {
	'test' : './Data/test_QyjYwdj.csv',
	'train':'./Data/train.csv',
	'submission':'./Data/sample_submission_Byiv0dS.csv',
	'coupon_item_mapping' :'./Data/coupon_item_mapping.csv',
	'campaign_data' : './Data/campaign_data.csv',
	'item_data' : './Data/item_data.csv',
	'customer_transaction_data':'./Data/customer_transaction_data.csv',
	'customer_demographics':'./Data/customer_demographics.csv',
}


In [83]:
train = pd.read_csv(file.get("train"))#
test = pd.read_csv(file.get("test"))#

coupon_item_mapping = pd.read_csv(file.get("coupon_item_mapping"))#No
item_data = pd.read_csv(file.get("item_data"))# may be yes
customer_transaction_data = pd.read_csv(file.get("customer_transaction_data"))#may be yes 

campaign_data = pd.read_csv(file.get("campaign_data"))#
customer_demographics = pd.read_csv(file.get("customer_demographics"))#
submission = pd.read_csv(file.get("submission"))


In [84]:
train.shape

(78369, 5)

In [85]:
data = pd.concat([train, test], sort=False).reset_index(drop = True)


In [86]:
ltr = len(train)

In [87]:
data = data.merge(campaign_data, on='campaign_id')#  campaign_data
data['start_date'] = pd.to_datetime(data['start_date'], dayfirst=True)
data['end_date'] = pd.to_datetime(data['end_date'], dayfirst=True)


In [88]:
data['campaign_type'].factorize()

(array([0, 0, 0, ..., 1, 1, 1]), Index(['X', 'Y'], dtype='object'))

In [89]:
data['campaign_type'] = pd.Series(data['campaign_type'].factorize()[0]).replace(-1, np.nan)

In [90]:
customer_demographics['no_of_children'] = customer_demographics['no_of_children'].replace('3+', 3).astype(float)
customer_demographics['family_size'] = customer_demographics['family_size'].replace('5+', 5).astype(float)
customer_demographics['marital_status'] = pd.Series(customer_demographics['marital_status'].factorize()[0]).replace(-1, np.nan)
customer_demographics['age_range'] = pd.Series(customer_demographics['age_range'].factorize()[0]).replace(-1, np.nan)

In [91]:
# use train data itself
redeemed_before_count = train.groupby("customer_id")['redemption_status'].sum().to_dict()
data['no_of_times_redeemed_before'] = data['customer_id'].map(redeemed_before_count)

In [92]:
# rented
rented_mean = customer_demographics.groupby("customer_id")['rented'].mean().to_dict()
data['rented_mean'] = data['customer_id'].map(rented_mean)

In [93]:
# income_bracket
income_bracket_sum = customer_demographics.groupby("customer_id")['income_bracket'].sum().to_dict()
data['income_bracket_sum'] = data['customer_id'].map(income_bracket_sum)

In [95]:
# age_range
age_range_mean = customer_demographics.groupby("customer_id")['age_range'].mean().to_dict()
data['age_range_mean'] = data['customer_id'].map(age_range_mean)


In [96]:
# family_size
family_size_mean = customer_demographics.groupby("customer_id")['family_size'].mean().to_dict()
data['family_size_mean'] = data['customer_id'].map(family_size_mean)

In [99]:
# no_of_children - actual number
no_of_children_mean = customer_demographics.groupby("customer_id")['no_of_children'].mean().to_dict()
data['no_of_children_mean'] = data['customer_id'].map(no_of_children_mean)
# actually represents if they have children or not
no_of_children_count = customer_demographics.groupby("customer_id")['no_of_children'].count().to_dict()
data['no_of_children_count'] = data['customer_id'].map(no_of_children_count)


In [100]:
# marital_status
marital_status_count = customer_demographics.groupby("customer_id")['marital_status'].count().to_dict()
data['marital_status_count'] = data['customer_id'].map(marital_status_count)


In [101]:
# customer_transaction_data
customer_transaction_data['date'] = pd.to_datetime(customer_transaction_data['date'])
# quantity	
quantity_mean = customer_transaction_data.groupby("customer_id")['quantity'].mean().to_dict()
data['quantity_mean'] = data['customer_id'].map(quantity_mean)
#coupon_discount
coupon_discount_mean = customer_transaction_data.groupby("customer_id")['coupon_discount'].mean().to_dict()
data['coupon_discount_mean'] = data['customer_id'].map(coupon_discount_mean)
# other_discount
other_discount_mean = customer_transaction_data.groupby("customer_id")['other_discount'].mean().to_dict()
data['other_discount_mean'] = data['customer_id'].map(other_discount_mean)
# day
customer_transaction_data['day'] = customer_transaction_data.date.dt.day
date_day_mean = customer_transaction_data.groupby("customer_id")['day'].mean().to_dict()
data['date_day_mean'] = data['customer_id'].map(date_day_mean)

# selling_price
selling_price_mean = customer_transaction_data.groupby("customer_id")['selling_price'].mean().to_dict()
data['selling_price_mean'] = data['customer_id'].map(selling_price_mean)
selling_price_mean = customer_transaction_data.groupby("customer_id")['selling_price'].sum().to_dict()
data['selling_price_sum'] = data['customer_id'].map(selling_price_mean)
selling_price_mean = customer_transaction_data.groupby("customer_id")['selling_price'].min().to_dict()
data['selling_price_min'] = data['customer_id'].map(selling_price_mean)
selling_price_mean = customer_transaction_data.groupby("customer_id")['selling_price'].max().to_dict()
data['selling_price_max'] = data['customer_id'].map(selling_price_mean)
selling_price_mean = customer_transaction_data.groupby("customer_id")['selling_price'].nunique().to_dict()
data['selling_price_nunique'] = data['customer_id'].map(selling_price_mean)

In [102]:
coupon_item_mapping = coupon_item_mapping.merge(item_data, how = 'left', on = 'item_id')
coupon_item_mapping['brand_type'] = pd.Series(coupon_item_mapping['brand_type'].factorize()[0]).replace(-1, np.nan)
coupon_item_mapping['category'] = pd.Series(coupon_item_mapping['category'].factorize()[0]).replace(-1, np.nan)

category = coupon_item_mapping.groupby("coupon_id")['category'].mean().to_dict()
data['category_mean'] = data['coupon_id'].map(category)
category = coupon_item_mapping.groupby("coupon_id")['category'].count().to_dict()
data['category_count'] = data['coupon_id'].map(category)
category = coupon_item_mapping.groupby("coupon_id")['category'].nunique().to_dict()
data['category_nunique'] = data['coupon_id'].map(category)
category = coupon_item_mapping.groupby("coupon_id")['category'].max().to_dict()
data['category_max'] = data['coupon_id'].map(category)
category = coupon_item_mapping.groupby("coupon_id")['category'].min().to_dict()
data['category_min'] = data['coupon_id'].map(category)

brand_mean = coupon_item_mapping.groupby("coupon_id")['brand'].mean().to_dict()
data['brand_mean'] = data['coupon_id'].map(brand_mean)
brand_mean = coupon_item_mapping.groupby("coupon_id")['brand'].count().to_dict()
data['brand_count'] = data['coupon_id'].map(brand_mean)
brand_mean = coupon_item_mapping.groupby("coupon_id")['brand'].min().to_dict()
data['brand_min'] = data['coupon_id'].map(brand_mean)
brand_mean = coupon_item_mapping.groupby("coupon_id")['brand'].max().to_dict()
data['brand_max'] = data['coupon_id'].map(brand_mean)
brand_mean = coupon_item_mapping.groupby("coupon_id")['brand'].nunique().to_dict()
data['brand_nunique'] = data['coupon_id'].map(brand_mean)


In [103]:
data.columns

Index(['id', 'campaign_id', 'coupon_id', 'customer_id', 'redemption_status',
       'campaign_type', 'start_date', 'end_date',
       'no_of_times_redeemed_before', 'rented_mean', 'income_bracket_sum',
       'age_range_mean', 'family_size_mean', 'no_of_children_mean',
       'no_of_children_count', 'marital_status_count', 'quantity_mean',
       'coupon_discount_mean', 'other_discount_mean', 'date_day_mean',
       'selling_price_mean', 'selling_price_sum', 'selling_price_min',
       'selling_price_max', 'selling_price_nunique', 'category_mean',
       'category_count', 'category_nunique', 'category_max', 'category_min',
       'brand_mean', 'brand_count', 'brand_min', 'brand_max', 'brand_nunique'],
      dtype='object')

In [104]:
data.shape

(128595, 35)

In [134]:
train_cols = ['campaign_id','coupon_id','campaign_type','rented_mean','income_bracket_sum','age_range_mean','family_size_mean',
 'no_of_children_mean',
 'no_of_children_count',
 'marital_status_count',
 'quantity_mean',
 'coupon_discount_mean',
 'other_discount_mean',
 'date_day_mean',
 'category_mean',
 'category_nunique',
 'category_max',
 'category_min',
 'brand_mean',
 'brand_max',
 'brand_nunique',
 'selling_price_mean',
 'selling_price_min',
 'selling_price_nunique']

In [135]:
len(train_cols)

24

In [136]:
train = data[data['redemption_status'].notnull()]
test = data[data['redemption_status'].isnull()]


In [137]:
train.head()

,id,campaign_id,coupon_id,customer_id,redemption_status,campaign_type,start_date,end_date,no_of_times_redeemed_before,rented_mean,...,category_mean,category_count,category_nunique,category_max,category_min,brand_mean,brand_count,brand_min,brand_max,brand_nunique
0,1,13,27,1053,0.0,0,2013-05-19,2013-07-05,0.0,0.0,...,0.0,125,1,0,0,1364.128,125,1105,1636,2
1,2,13,116,48,0.0,0,2013-05-19,2013-07-05,0.0,0.0,...,0.0,3,1,0,0,56.000,3,56,56,1
2,7,13,644,1050,0.0,0,2013-05-19,2013-07-05,0.0,NaN,...,0.0,4,1,0,0,611.000,4,611,611,1
3,21,13,1028,89,0.0,0,2013-05-19,2013-07-05,0.0,0.0,...,0.0,6,1,0,0,1639.000,6,1639,1639,1
4,23,13,517,1067,0.0,0,2013-05-19,2013-07-05,0.0,0.0,...,3.0,3,1,3,3,261.000,3,261,261,1


In [138]:
train.isna().sum()

id                                 0
campaign_id                        0
coupon_id                          0
customer_id                        0
redemption_status                  0
campaign_type                      0
start_date                         0
end_date                           0
no_of_times_redeemed_before        0
rented_mean                    34708
income_bracket_sum             34708
age_range_mean                 34708
family_size_mean               34708
no_of_children_mean            64313
no_of_children_count           34708
marital_status_count           34708
quantity_mean                      0
coupon_discount_mean               0
other_discount_mean                0
date_day_mean                      0
selling_price_mean                 0
selling_price_sum                  0
selling_price_min                  0
selling_price_max                  0
selling_price_nunique              0
category_mean                      0
category_count                     0
c

In [110]:
test.isna().sum()

id                                 0
campaign_id                        0
coupon_id                          0
customer_id                        0
redemption_status              50226
campaign_type                      0
start_date                         0
end_date                           0
no_of_times_redeemed_before     4018
rented_mean                    19287
income_bracket_sum             19287
age_range_mean                 19287
family_size_mean               19287
no_of_children_mean            40064
no_of_children_count           19287
marital_status_count           19287
quantity_mean                      0
coupon_discount_mean               0
other_discount_mean                0
date_day_mean                      0
selling_price_mean                 0
selling_price_sum                  0
selling_price_min                  0
selling_price_max                  0
selling_price_nunique              0
category_mean                      0
category_count                     0
c

In [52]:
pd.get_dummies(data[train_cols].fillna(0), columns=train_cols, drop_first=True, sparse=True).columns

Index(['campaign_id_2', 'campaign_id_3', 'campaign_id_4', 'campaign_id_5',
       'campaign_id_6', 'campaign_id_7', 'campaign_id_8', 'campaign_id_9',
       'campaign_id_10', 'campaign_id_11',
       ...
       'selling_price_nunique_408', 'selling_price_nunique_423',
       'selling_price_nunique_430', 'selling_price_nunique_437',
       'selling_price_nunique_439', 'selling_price_nunique_444',
       'selling_price_nunique_451', 'selling_price_nunique_452',
       'selling_price_nunique_500', 'selling_price_nunique_536'],
      dtype='object', length=9978)

In [61]:
train.columns

Index(['id', 'campaign_id', 'coupon_id', 'customer_id', 'redemption_status',
       'campaign_type', 'start_date', 'end_date', 'rented_mean',
       'income_bracket_sum', 'age_range_mean', 'family_size_mean',
       'no_of_children_mean', 'no_of_children_count', 'marital_status_count',
       'quantity_mean', 'coupon_discount_mean', 'other_discount_mean',
       'date_day_mean', 'selling_price_mean', 'selling_price_sum',
       'selling_price_min', 'selling_price_max', 'selling_price_nunique',
       'category_mean', 'category_count', 'category_nunique', 'category_max',
       'category_min', 'brand_mean', 'brand_count', 'brand_min', 'brand_max',
       'brand_nunique'],
      dtype='object')

In [55]:
train.shape

(78369, 34)

In [57]:
test.shape

(50226, 34)

In [58]:
target.shape

(78369,)

In [139]:
train_np = train[train_cols].fillna(0).values
test_np = test[train_cols].fillna(0).values

In [146]:
def run_cv_model(train, test, target, model_fn, params={}, eval_fn=None, label='model'):
    kf = StratifiedKFold(n_splits=5, shuffle = True, random_state = 228)
    fold_splits = kf.split(train, target)
    cv_scores = []
    pred_full_test = 0
    pred_train = np.zeros((train.shape[0]))
    i = 1
    for dev_index, val_index in fold_splits:
        print('Started ' + label + ' fold ' + str(i) + '/5')
        dev_X, val_X = train[dev_index], train[val_index]
        dev_y, val_y = target[dev_index], target[val_index]
        params2 = params.copy()
        pred_val_y, pred_test_y = model_fn(dev_X, dev_y, val_X, val_y, test, params2)
        pred_full_test = pred_full_test + pred_test_y
        pred_train[val_index] = pred_val_y
        if eval_fn is not None:
            cv_score = eval_fn(val_y, pred_val_y)
            cv_scores.append(cv_score)
            print(label + ' cv score {}: {}'.format(i, cv_score))
        i += 1
    print('{} cv scores : {}'.format(label, cv_scores))
    print('{} cv mean score : {}'.format(label, np.mean(cv_scores)))
    print('{} cv std score : {}'.format(label, np.std(cv_scores)))
    pred_full_test = pred_full_test / 10.0
    results = {'label': label,
              'train': pred_train, 'test': pred_full_test,
              'cv': cv_scores}
    return results


def runLR(train_X, train_y, test_X, test_y, test_X2, params):
    print('Train LR')
    model = LogisticRegression(**params)
    model.fit(train_X, train_y)
    print('Predict 1/2')
    pred_test_y = logit(model.predict_proba(test_X)[:, 1])
    print('Predict 2/2')
    pred_test_y2 = logit(model.predict_proba(test_X2)[:, 1])
    return pred_test_y, pred_test_y2

In [67]:
target = train['redemption_status'].values
lr_params = {'solver': 'lbfgs','C': 1.8,'max_iter' : 2000}
results = run_cv_model(train_np, test_np, target, runLR, lr_params, auc, 'lr')

Started lr fold 1/5
Train LR


/home/ubuntu/miniconda3/envs/xray/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


Predict 1/2
Predict 2/2
lr cv score 1: 0.8174433458251289
Started lr fold 2/5
Train LR


/home/ubuntu/miniconda3/envs/xray/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


Predict 1/2
Predict 2/2
lr cv score 2: 0.8208574170918818
Started lr fold 3/5
Train LR


...

/home/ubuntu/miniconda3/envs/xray/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


Predict 1/2
Predict 2/2
lr cv score 8: 0.8351435850747744
Started lr fold 9/5
Train LR


/home/ubuntu/miniconda3/envs/xray/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


Predict 1/2
Predict 2/2
lr cv score 9: 0.8019750446387612
Started lr fold 10/5
Train LR
Predict 1/2
Predict 2/2
lr cv score 10: 0.8494189707481825
lr cv scores : [0.8174433458251289, 0.8208574170918818, 0.8195853006147095, 0.8672676137847318, 0.810428179232566, 0.858639805777279, 0.8201481371697966, 0.8351435850747744, 0.8019750446387612, 0.8494189707481825]
lr cv mean score : 0.8300907399957811
lr cv std score : 0.020561663135559557


/home/ubuntu/miniconda3/envs/xray/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


In [68]:
tmp = dict(zip(test.id.values, results['test']))
answer1 = pd.DataFrame()
answer1['id'] = test.id.values
answer1['redemption_status'] = answer1['id'].map(tmp)
answer1.to_csv('submit_new.csv', index = None)

**xgboost**

In [113]:
def runXgb(train_X, train_y, test_X, test_y, test_X2, params):
    print('Train LR')
    model = XGBClassifier(random_state=42)
    model.fit(train_X, train_y)
    print('Predict 1/2')
    pred_test_y = logit(model.predict_proba(test_X)[:, 1])
    print('Predict 2/2')
    pred_test_y2 = logit(model.predict_proba(test_X2)[:, 1])
    return pred_test_y, pred_test_y2

In [71]:
target = train['redemption_status'].values
lr_params = {'solver': 'lbfgs','C': 1.8,'max_iter' : 2000}
results = run_cv_model(train_np, test_np, target, runXgb, lr_params, auc, 'xgb')

Started lr fold 1/5
Train LR
Predict 1/2
Predict 2/2
lr cv score 1: 0.9196237640532701
Started lr fold 2/5
Train LR
Predict 1/2
Predict 2/2
lr cv score 2: 0.9069978756889896
Started lr fold 3/5
Train LR
Predict 1/2
Predict 2/2
lr cv score 3: 0.9192505981241135
Started lr fold 4/5
Train LR
Predict 1/2
Predict 2/2
lr cv score 4: 0.9319179493694114
Started lr fold 5/5
Train LR
Predict 1/2
Predict 2/2
lr cv score 5: 0.8962757510956787
Started lr fold 6/5
Train LR
Predict 1/2
Predict 2/2
lr cv score 6: 0.9253500525784619
Started lr fold 7/5
Train LR
Predict 1/2
Predict 2/2
lr cv score 7: 0.9145855829151758
Started lr fold 8/5
Train LR
Predict 1/2
Predict 2/2
lr cv score 8: 0.9036667301842716
Started lr fold 9/5
Train LR
Predict 1/2
Predict 2/2
lr cv score 9: 0.9028039493835263
Started lr fold 10/5
Train LR
Predict 1/2
Predict 2/2
lr cv score 10: 0.9265609794493103
lr cv scores : [0.9196237640532701, 0.9069978756889896, 0.9192505981241135, 0.9319179493694114, 0.8962757510956787, 0.9253500525

In [72]:
tmp = dict(zip(test.id.values, results['test']))
answer1 = pd.DataFrame()
answer1['id'] = test.id.values
answer1['redemption_status'] = answer1['id'].map(tmp)
answer1.to_csv('submit_new_xgb.csv', index = None)

In [114]:
target = train['redemption_status'].values
lr_params = {'solver': 'lbfgs','C': 1.8,'max_iter' : 2000}
results = run_cv_model(train_np, test_np, target, runXgb, lr_params, auc, 'xgb')

Started xgb fold 1/5
Train LR
Predict 1/2
Predict 2/2
xgb cv score 1: 0.9604805812566606
Started xgb fold 2/5
Train LR
Predict 1/2
Predict 2/2
xgb cv score 2: 0.9641646023445054
Started xgb fold 3/5
Train LR
Predict 1/2
Predict 2/2
xgb cv score 3: 0.9680991650963703
Started xgb fold 4/5
Train LR
Predict 1/2
Predict 2/2
xgb cv score 4: 0.9692248382065451
Started xgb fold 5/5
Train LR
Predict 1/2
Predict 2/2
xgb cv score 5: 0.9586562144918944
Started xgb fold 6/5
Train LR
Predict 1/2
Predict 2/2
xgb cv score 6: 0.9692371888519546
Started xgb fold 7/5
Train LR
Predict 1/2
Predict 2/2
xgb cv score 7: 0.9671155244084041
Started xgb fold 8/5
Train LR
Predict 1/2
Predict 2/2
xgb cv score 8: 0.9604417649425167
Started xgb fold 9/5
Train LR
Predict 1/2
Predict 2/2
xgb cv score 9: 0.9643022238219248
Started xgb fold 10/5
Train LR
Predict 1/2
Predict 2/2
xgb cv score 10: 0.969627447192169
xgb cv scores : [0.9604805812566606, 0.9641646023445054, 0.9680991650963703, 0.9692248382065451, 0.9586562144

In [115]:
tmp = dict(zip(test.id.values, results['test']))
answer1 = pd.DataFrame()
answer1['id'] = test.id.values
answer1['redemption_status'] = answer1['id'].map(tmp)
answer1.to_csv('submit_new_xgb_with_my_inception_feature.csv', index = None)

### Bayesian Optimisation

In [140]:
dtrain = xgb.DMatrix(train_np, label=target)

In [141]:
train_np.shape

(78369, 24)

In [142]:
def bo_tune_xgb(max_depth, gamma, n_estimators ,learning_rate, subsample):
    params = {'max_depth': int(max_depth),
              'gamma': gamma,
              'n_estimators': int(n_estimators),
              'learning_rate':learning_rate,
              'subsample': float(subsample),
              'eta': 0.1,
               'objective':'binary:logistic',
              'eval_metric': 'auc'}
    #Cross validating with the specified parameters in 5 folds and 70 iterations
    cv_result = xgb.cv(params, dtrain, num_boost_round=70, nfold=5)
    #Return the negative RMSE
    return cv_result['test-auc-mean'].iloc[-1]

In [143]:
xgb_bo = BayesianOptimization(bo_tune_xgb, {'max_depth': (3, 10), 
                                             'gamma': (0, 1),
                                             'learning_rate':(0,1),
                                             'n_estimators':(100,120),
                                            'subsample':(0.1,0.3)
                                            })

In [144]:
xgb_bo.maximize(n_iter=5, init_points=8, acq='ei')

|   iter    |  target   |   gamma   | learni... | max_depth | n_esti... | subsample |
-------------------------------------------------------------------------------------
|  1        |  0.8971   |  0.4825   |  0.4488   |  4.53     |  117.1    |  0.2903   |
|  2        |  0.833    |  0.04989  |  0.9978   |  3.954    |  113.9    |  0.1999   |
|  3        |  0.8611   |  0.2141   |  0.4501   |  7.414    |  108.3    |  0.1594   |
|  4        |  0.8727   |  0.2158   |  0.6634   |  3.655    |  119.4    |  0.1952   |
|  5        |  0.8993   |  0.2872   |  0.4883   |  3.239    |  113.1    |  0.2781   |
|  6        |  0.8809   |  0.3213   |  0.4004   |  8.183    |  117.6    |  0.2675   |
|  7        |  0.9074   |  0.923    |  0.1733   |  3.168    |  106.2    |  0.1987   |
|  8        |  0.8323   |  0.7869   |  0.8965   |  7.656    |  119.3    |  0.2554   |
|  9        |  0.5      |  1.0      |  0.0      |  3.0      |  115.3    |  0.3      |
|  10       |  0.5      |  0.0      |  0.0      |  3.0

In [131]:
def runXgbHighest(train_X, train_y, test_X, test_y, test_X2, params):
    print('Train LR')
    model = XGBClassifier(random_state=42, learning_rate=0.03455, gamma=0.1887, max_depth=8, n_estimators=116, subsample=0.2643)
    model.fit(train_X, train_y)
    print('Predict 1/2')
    pred_test_y = logit(model.predict_proba(test_X)[:, 1])
    print('Predict 2/2')
    pred_test_y2 = logit(model.predict_proba(test_X2)[:, 1])
    return pred_test_y, pred_test_y2

In [132]:
target = train['redemption_status'].values
lr_params = {'solver': 'lbfgs','C': 1.8,'max_iter' : 2000}
results = run_cv_model(train_np, test_np, target, runXgbHighest, lr_params, auc, 'xgb-bpo')

Started xgb-bpo fold 1/5
Train LR
Predict 1/2
Predict 2/2
xgb-bpo cv score 1: 0.9554970958339507
Started xgb-bpo fold 2/5
Train LR
Predict 1/2
Predict 2/2
xgb-bpo cv score 2: 0.9634103307855716
Started xgb-bpo fold 3/5
Train LR
Predict 1/2
Predict 2/2
xgb-bpo cv score 3: 0.9661248262087753
Started xgb-bpo fold 4/5
Train LR
Predict 1/2
Predict 2/2
xgb-bpo cv score 4: 0.9712794562893016
Started xgb-bpo fold 5/5
Train LR
Predict 1/2
Predict 2/2
xgb-bpo cv score 5: 0.9562954768407755
Started xgb-bpo fold 6/5
Train LR
Predict 1/2
Predict 2/2
xgb-bpo cv score 6: 0.9679112588483553
Started xgb-bpo fold 7/5
Train LR
Predict 1/2
Predict 2/2
xgb-bpo cv score 7: 0.9626587057935113
Started xgb-bpo fold 8/5
Train LR
Predict 1/2
Predict 2/2
xgb-bpo cv score 8: 0.959531345938049
Started xgb-bpo fold 9/5
Train LR
Predict 1/2
Predict 2/2
xgb-bpo cv score 9: 0.9605696823413999
Started xgb-bpo fold 10/5
Train LR
Predict 1/2
Predict 2/2
xgb-bpo cv score 10: 0.9698125965996909
xgb-bpo cv scores : [0.955497

In [133]:
tmp = dict(zip(test.id.values, results['test']))
answer1 = pd.DataFrame()
answer1['id'] = test.id.values
answer1['redemption_status'] = answer1['id'].map(tmp)
answer1.to_csv('submit_new_xgb_with_my_inception_feature_BPO.csv', index = None)

In [150]:
def runXgbHighest2(train_X, train_y, test_X, test_y, test_X2, params):
    print('Train LR')
    model = XGBClassifier(random_state=42, learning_rate=0.1733, gamma=0.923, max_depth=3, n_estimators=106, subsample=0.1987)
    model.fit(train_X, train_y)
    print('Predict 1/2')
    pred_test_y = logit(model.predict_proba(test_X)[:, 1])
    print('Predict 2/2')
    pred_test_y2 = logit(model.predict_proba(test_X2)[:, 1])
    print('feature importance : {}'.format(model.feature_importances_))
    return pred_test_y, pred_test_y2

In [151]:
target = train['redemption_status'].values
lr_params = {'solver': 'lbfgs','C': 1.8,'max_iter' : 2000}
results = run_cv_model(train_np, test_np, target, runXgbHighest2, lr_params, auc, 'xgb-bpo2')

Started xgb-bpo2 fold 1/5
Train LR
Predict 1/2
Predict 2/2
feature importance : [0.05037954 0.04591098 0.05606027 0.03044404 0.04609721 0.0337368
 0.05608289 0.03338706 0.         0.0485968  0.04821579 0.05407567
 0.03965615 0.03938425 0.06306253 0.02905087 0.03927152 0.03638927
 0.05644433 0.04014567 0.03124542 0.04412053 0.03865521 0.03958711]
xgb-bpo2 cv score 1: 0.9153043463685573
Started xgb-bpo2 fold 2/5
Train LR
Predict 1/2
Predict 2/2
feature importance : [0.05307731 0.04672448 0.05680313 0.0479537  0.03344129 0.04509368
 0.02907523 0.         0.         0.03389921 0.04171652 0.05706872
 0.03926303 0.04610841 0.047458   0.04981609 0.05788095 0.04451828
 0.05359478 0.04155997 0.04086689 0.04405981 0.0406845  0.0493359 ]
xgb-bpo2 cv score 2: 0.9277235819694692
Started xgb-bpo2 fold 3/5
Train LR
Predict 1/2
Predict 2/2
feature importance : [0.05655163 0.04777016 0.06292161 0.0517264  0.04835346 0.03664533
 0.03281117 0.         0.         0.05184841 0.04450717 0.05238169
 0.037626

In [149]:
tmp = dict(zip(test.id.values, results['test']))
answer1 = pd.DataFrame()
answer1['id'] = test.id.values
answer1['redemption_status'] = answer1['id'].map(tmp)
answer1.to_csv('submit_new_xgb_BPO_2.csv', index = None)

In [148]:
def runXgbHighest3(train_X, train_y, test_X, test_y, test_X2, params):
    print('Train LR')
    model = XGBClassifier(random_state=42, learning_rate=0.4501, gamma=0.2141, max_depth=7, n_estimators=108, subsample=0.1594)
    model.fit(train_X, train_y)
    print('Predict 1/2')
    pred_test_y = logit(model.predict_proba(test_X)[:, 1])
    print('Predict 2/2')
    pred_test_y2 = logit(model.predict_proba(test_X2)[:, 1])
    return pred_test_y, pred_test_y2

**Tetsing sanity of data**

In [35]:
item_data.head()

,item_id,brand,brand_type,category
0,1,1,Established,Grocery
1,2,1,Established,Miscellaneous
2,3,56,Local,Bakery
3,4,56,Local,Grocery
4,5,56,Local,Grocery


In [36]:
coupon_item_mapping.head()

,coupon_id,item_id
0,105,37
1,107,75
2,494,76
3,522,77
4,518,77


In [39]:
coupon_item_mapping.shape

(92663, 2)

In [38]:
coupon_item_mapping.merge(item_data, how = 'left', on = 'item_id').shape

(92663, 5)

In [30]:
data.head()

,id,campaign_id,coupon_id,customer_id,redemption_status,campaign_type,start_date,end_date,rented_mean,income_bracket_sum,age_range_mean,family_size_mean,no_of_children_mean,no_of_children_count
0,1,13,27,1053,0.0,0,2013-05-19,2013-07-05,0.0,5.0,1.0,1.0,NaN,0.0
1,2,13,116,48,0.0,0,2013-05-19,2013-07-05,0.0,3.0,3.0,2.0,NaN,0.0
2,7,13,644,1050,0.0,0,2013-05-19,2013-07-05,NaN,NaN,NaN,NaN,NaN,NaN
3,21,13,1028,89,0.0,0,2013-05-19,2013-07-05,0.0,3.0,1.0,1.0,NaN,0.0
4,23,13,517,1067,0.0,0,2013-05-19,2013-07-05,0.0,5.0,1.0,1.0,NaN,0.0


In [12]:
ltr

78369

In [6]:
data.shape

(128595, 5)

In [7]:
train.shape

(78369, 5)

In [8]:
test.shape

(50226, 4)

In [9]:
data.head()

,id,campaign_id,coupon_id,customer_id,redemption_status
0,1,13,27,1053,0.0
1,2,13,116,48,0.0
2,6,9,635,205,0.0
3,7,13,644,1050,0.0
4,9,8,1017,1489,0.0


In [10]:
data.isna().sum()

id                       0
campaign_id              0
coupon_id                0
customer_id              0
redemption_status    50226
dtype: int64